<a href="https://colab.research.google.com/github/SumeetChougule/PM-HR/blob/main/CNN_trial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/DifferentiableUniverseInitiative/flowpm.git

In [ ]:
!pip install git+https://github.com/DifferentiableUniverseInitiative/flowpm.git

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras.layers as tfl
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
from tensorflow.keras import datasets, layers, models, losses

import tensorflow_probability as tfp
tfd = tfp.distributions

# Input data from PM



In [6]:
bs, nc = 100, 32
nsteps = 5
a0, af, nsteps = 0.1, 1.0,  nsteps
stages = np.linspace(a0, af, nsteps, endpoint=True)
donbody = False
dnoise = 1. #0.1

klin, plin = np.loadtxt('flowpm/flowpm/data/Planck15_a1p00.txt').T
ipklin = iuspline(klin, plin)

In [7]:
@tf.function
def pm(linear):
    print("PM graph")
    cosmo = flowpm.cosmology.Planck15()
    state = lpt_init(cosmo, linear, a=a0, order=2)
    final_state = nbody(cosmo, state,  stages, nc)
    tfinal_field = cic_paint(tf.zeros_like(linear), final_state[0])
    return final_state, tfinal_field

In [8]:
ic = linear_field(nc, bs, ipklin, name='pm').numpy()
state, fin = pm(tf.constant(ic))
data = fin + np.random.normal(0, dnoise, nc**3).reshape(fin.shape)

data = data.numpy().astype(np.float32)

PM graph


In [2]:
def cnn_model():
    model = tf.keras.Sequential([
        tfl.InputLayer(input_shape=(32,32,3)),
        tfl.Conv2D(filters = 32, kernel_size= 3, strides=(2,2), padding='same', activation= 'relu'),
        tfl.MaxPooling2D(pool_size=(2,2), strides = None, padding ='valid'),
        tfl.Conv2D(filters = 64, kernel_size= 3, strides=(2,2), padding='same', activation= 'relu'),
        tfl.Flatten(),
        tfl.Dense(80, activation= 'relu'),
        tfl.Dense(30, activation = 'relu')
        ])
    return model

cnn = cnn_model()

In [3]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 16, 16, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 32)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 64)          18496     
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 80)                82000     
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                        

In [4]:
cnn.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

